In [5]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import requests
import pandas as pd
import json

In [8]:
class Pokemon():
    base_url = "https://pokeapi.co/api/v2/pokemon/?offset=0&limit=150/"
    nat_url = "https://pokeapi.co/api/v2/nature"
    pokedex = {
        'nome': [],
        'habilidade': [],
        'local': [],
        'tipo': [],
        'hp':[],
        'ataque': [],
        'defesa': []
        }
    
    def get_pokemons(self):
        response = requests.request("GET", self.base_url)
        cont = 1
        attack = None
        if response:
            response = response.json()
            for poke in response['results']:
                self.pokedex['nome'].append(poke['name'])
                response_detail = requests.request("GET", poke['url'])
                self.pokedex['habilidade'].append(response_detail.json()['abilities'][0]['ability']['name'])
                location = requests.request("GET", response_detail.json()['location_area_encounters'])
                try:
                  self.pokedex['local'].append(location.json()[0]['location_area']['name'])
                except:
                   self.pokedex['local'].append("Não Informado")
                self.pokedex['tipo'].append(response_detail.json()['types'][0]['type']['name'])
                nature = requests.request("GET",f"{self.nat_url}/{cont}")
                cont+= 1
                try:
                  self.pokedex['hp'].append(nature.json()['move_battle_style_preferences'][0]['high_hp_preference'])
                except:
                  self.pokedex['hp'].append("Não Informado")
                try:
                  attack = requests.request("GET",nature.json()['decreased_stat']['url'])
                  if attack:
                    self.pokedex['ataque'].append(attack.json()['affecting_moves']['decrease'][0]['move']['name'])
                    self.pokedex['defesa'].append(attack.json()['affecting moves']['increase'][0]['move']['name'])
                except:
                  self.pokedex['ataque'].append("Não Informado")
                  self.pokedex['defesa'].append("Não Informado")

                
            # print(attack.json()['affecting_moves'])
            return self.pokedex

In [9]:
x = Pokemon().get_pokemons()

In [24]:
new_data = pd.DataFrame.from_dict(x['nome'])
new_data['Habilidade'] = pd.DataFrame.from_dict(x['habilidade'])
new_data['Local'] = pd.DataFrame.from_dict(x['local'])
new_data['Tipo'] = pd.DataFrame.from_dict(x['tipo'])
new_data['HP'] = pd.DataFrame.from_dict(x['hp'])
new_data['Ataque'] = pd.DataFrame.from_dict(x['ataque'])
new_data['Defesa'] = pd.DataFrame.from_dict(x['defesa'])

In [31]:
new_data = new_data.rename(columns={0:"Nome"})

In [35]:
new_data.to_excel("pokedex.xlsx",index=False)